In [2]:
import tensorflow as tf

from tensorflow.keras import layers

import numpy as np

2025-03-06 17:08:08.824719: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-06 17:08:08.828741: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-06 17:08:08.841305: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741291688.861140   20675 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741291688.867637   20675 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-06 17:08:08.886394: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [3]:
opponent_hist = "RRRPPPSSS" * 10
player_hist = "RPS" * 30
player_hist

'RPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPS'

In [4]:
vocab = sorted(set(opponent_hist))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return [char2idx[c] for c in text]

def int_to_text(ints):
  return ''.join(idx2char[ints])

In [6]:
length = 20
next_data = []
data = []

for i in range(len(opponent_hist) - length): #list with next play
    next_data.append(opponent_hist[length + i])

for i in range(len(opponent_hist) - length): #list with previous 'length' plays
    data.append(opponent_hist[i : length + i] + player_hist[i : length + i])

k = 0
print(' ', len(next_data), len(next_data[k]), next_data[k], '\n ',
      len(data), len(data[k]), data[k])

  70 1 R 
  70 40 RRRPPPSSSRRRPPPSSSRRRPSRPSRPSRPSRPSRPSRP


In [80]:
encoded_data = np.array([text_to_int(d) for d in data])
encoded_next = np.array([text_to_int(d)[0] for d in next_data])
print(encoded_data.shape, encoded_next.shape)

(70, 40) (70,)


In [81]:
features_onehot = np.array([tf.keras.utils.to_categorical(feature, num_classes=len(vocab)) for feature in encoded_data])
labels_onehot = tf.keras.utils.to_categorical(encoded_next, num_classes=len(vocab))
features_onehot.shape, labels_onehot.shape

((70, 40, 3), (70, 3))

In [82]:
model = tf.keras.Sequential([
    layers.LSTM(128, input_shape=(features_onehot.shape[1], features_onehot.shape[2])),
    layers.Dense(127, activation='relu'),
    layers.Dense(3, activation='softmax')
])

model.compile(loss='CategoricalCrossentropy',
              optimizer='adam',
              metrics=['Accuracy'])

history = model.fit(
    features_onehot,
    labels_onehot,
    epochs=100)

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - Accuracy: 0.2731 - loss: 1.1033
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - Accuracy: 0.2770 - loss: 1.1034
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - Accuracy: 0.3251 - loss: 1.0985
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - Accuracy: 0.3283 - loss: 1.0974
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - Accuracy: 0.4227 - loss: 1.0979
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - Accuracy: 0.3550 - loss: 1.0965
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - Accuracy: 0.3121 - loss: 1.1001
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - Accuracy: 0.3667 - loss: 1.0962
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - Accuracy: 0.3205 - loss: 1.0972
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - Accuracy: 0.3472 - loss: 1.0947
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - Accuracy: 0.3003 - loss: 1.0979
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - Accuracy: 0.3316 - lo

In [92]:
def predict(opponent, player):
    # input: opponent and player histories
    # output: predicted output and it's probability
    m = min(len(opponent), len(player)) #if the inputs have different lengths, 'm' is the smaller length,
    seq = opponent[-m :] + player[-m :] #only use the last 'm' plays of each player
    seq_encoded = np.array([text_to_int(seq)]) #encode sequence of chars into sequence of ints
    seq_onehot = np.array([tf.keras.utils.to_categorical(se, num_classes=len(vocab)) for se in seq_encoded])
        #encode sequence of ints into sequence of one-hots
    pred = model.predict(seq_onehot) #run the prediction. Return list of probabilities for next char
    idx = np.argmax(pred) #idx of the char with max probability, equal to the integer encoding for the chars
    return int_to_text(idx), round(float(pred[0][idx]), 2) #predicted char, probability

predict('RRRPPPSSS', 'RPSRPSR')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


('R', 0.53)